In [0]:
import sys, os, time
import matplotlib.pyplot as plt
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Importing data

In [0]:
train_file = np.load("/content/gdrive/My Drive/Colab Notebooks/ELEC97052 Selected Topics in Machine Learning/CW3/assets/train.npz", allow_pickle=True)
train_X, train_Y, action_names = train_file['x'], train_file['y'], train_file['action_names']

test_file = np.load("/content/gdrive/My Drive/Colab Notebooks/ELEC97052 Selected Topics in Machine Learning/CW3/assets/test.npz", allow_pickle=True)
test_X, test_Y = test_file['x'], test_file['y']

# information about first element in training dataset
input = train_X[0] # [N, 21*3]
output_label = train_Y[0] # starts at 1!
action_name = action_names[output_label - 1]

In [0]:
print(type(action_name))

<class 'numpy.ndarray'>
ERROR! Session/line number was not unique in database. History logging moved to new session 59


Max number of timesteps

In [0]:
max = 0
for i in range(0,950):
  input = train_X[i]
  if input.shape[0] > max:
    max = input.shape[0]

print(max)

1151


Zero padding (actually -99 padding)

In [0]:
x_train = -99*np.ones(shape=(train_X.shape[0], 21, 1151, 3), dtype=float)
for i in range(0, train_X.shape[0]):
  prepose = train_X[i]
  pose = prepose.reshape(prepose.shape[0], 21, 3)
  for j in range(0, pose.shape[0]):
    for k in range(0, pose.shape[1]):
      for l in range(0, pose.shape[2]):
        x_train[i][k][j][l] = pose[j][k][l]

x_test = -99*np.ones(shape=(test_X.shape[0], 21, 1151, 3), dtype=float)
for i in range(0, test_X.shape[0]):
  prepose = test_X[i]
  pose = prepose.reshape(prepose.shape[0], 21, 3)
  for j in range(0, pose.shape[0]):
    for k in range(0, pose.shape[1]):
      for l in range(0, pose.shape[2]):
        x_test[i][k][j][l] = pose[j][k][l]

y_train = to_categorical(train_Y-1, num_classes=45)
y_test = to_categorical(test_Y-1, num_classes=45)

T = np.concatenate((x_train[:,0,:,:],x_train[:,1,:,:],x_train[:,6,:,:],x_train[:,7,:,:],x_train[:,8,:,:]), axis=-1)
I = np.concatenate((x_train[:,0,:,:],x_train[:,2,:,:],x_train[:,9,:,:],x_train[:,10,:,:],x_train[:,11,:,:]),axis=-1)
M = np.concatenate((x_train[:,0,:,:],x_train[:,3,:,:],x_train[:,12,:,:],x_train[:,13,:,:],x_train[:,14,:,:]),axis=-1)
R = np.concatenate((x_train[:,0,:,:],x_train[:,4,:,:],x_train[:,15,:,:],x_train[:,16,:,:],x_train[:,17,:,:]),axis=-1)
P = np.concatenate((x_train[:,0,:,:],x_train[:,5,:,:],x_train[:,18,:,:],x_train[:,19,:,:],x_train[:,20,:,:]),axis=-1)

Tout = np.concatenate((x_test[:,0,:,:],x_test[:,1,:,:],x_test[:,6,:,:],x_test[:,7,:,:],x_test[:,8,:,:]), axis=-1)
Iout = np.concatenate((x_test[:,0,:,:],x_test[:,2,:,:],x_test[:,9,:,:],x_test[:,10,:,:],x_test[:,11,:,:]),axis=-1)
Mout = np.concatenate((x_test[:,0,:,:],x_test[:,3,:,:],x_test[:,12,:,:],x_test[:,13,:,:],x_test[:,14,:,:]),axis=-1)
Rout = np.concatenate((x_test[:,0,:,:],x_test[:,4,:,:],x_test[:,15,:,:],x_test[:,16,:,:],x_test[:,17,:,:]),axis=-1)
Pout = np.concatenate((x_test[:,0,:,:],x_test[:,5,:,:],x_test[:,18,:,:],x_test[:,19,:,:],x_test[:,20,:,:]),axis=-1)

print(x_train.shape)
print(x_test.shape)
print(T.shape)

(950, 21, 1151, 3)
(225, 21, 1151, 3)
(950, 1151, 15)


In [0]:
pose = train_X[i]
print(pose[10][10])
print("------------------------")
newpose = pose.reshape(pose.shape[0], 21, 3)
print(newpose[10][3][1])
print(newpose.shape)

-0.36706489620972604
------------------------
-0.36706489620972604
(42, 21, 3)


# **Model**

In [0]:
def create_model(lr, lstm, outsize): 
  Tin = layers.Input(shape=(1151, 15))
  Iin = layers.Input(shape=(1151, 15))
  Min = layers.Input(shape=(1151, 15))
  Rin = layers.Input(shape=(1151, 15))
  Pin = layers.Input(shape=(1151, 15))
     
  # T = tf.gather(samples, [0,1,6,7,8], axis=1)
  # I = tf.gather(samples, [0,2,9,10,11], axis=1)
  # M = tf.gather(samples, [0,3,12,13,14], axis=1)
  # R = tf.gather(samples, [0,4,15,16,17], axis=1)
  # P = tf.gather(samples, [0,5,18,19,20], axis=1)
 
  T = layers.Masking(mask_value=-99)(Tin)
  encodedTd = layers.Bidirectional(layers.GRU(lstm, return_sequences=True))(T)
  encodedT = layers.Dropout(0.2)(encodedTd)

  I = layers.Masking(mask_value=-99)(Iin)
  encodedId = layers.Bidirectional(layers.GRU(lstm, return_sequences=True))(I)
  encodedI = layers.Dropout(0.2)(encodedId)

  M = layers.Masking(mask_value=-99)(Min)
  encodedMd = layers.Bidirectional(layers.GRU(lstm, return_sequences=True))(M)
  encodedM = layers.Dropout(0.2)(encodedMd)

  R = layers.Masking(mask_value=-99)(Rin)
  encodedRd = layers.Bidirectional(layers.GRU(lstm, return_sequences=True))(R)
  encodedR = layers.Dropout(0.2)(encodedRd)

  P = layers.Masking(mask_value=-99)(Pin)
  encodedPd = layers.Bidirectional(layers.GRU(lstm, return_sequences=True))(P)
  encodedP = layers.Dropout(0.2)(encodedPd)

  #hand = tf.stack([encodedT, encodedI, encodedM, encodedR, encodedP],1)
  hand = layers.Concatenate()([encodedT, encodedI, encodedM, encodedR, encodedP])
  encodedhand = layers.Bidirectional(layers.GRU(outsize))(hand)
  encodedhand_dropout = layers.Dropout(0.2)(encodedhand)
  prediction = layers.Dense(45, activation = 'softmax')(encodedhand_dropout)
  model = models.Model(inputs=[Tin,Iin,Min,Rin,Pin], outputs=prediction)

  adam = tf.keras.optimizers.Adam(learning_rate=lr)
  model.compile(optimizer=adam, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
  return model

In [0]:
y = layers.Masking(mask_value=-99)(x_train)
print("y:")
print(y[0][15][50][2])
T = layers.Concatenate()([y[:,0,:,:],y[:,1,:,:],y[:,6,:,:],y[:,7,:,:],y[:,8,:,:]])
I = layers.Concatenate()([y[:,0,:,:],y[:,2,:,:],y[:,9,:,:],y[:,10,:,:],y[:,11,:,:]])
M = layers.Concatenate()([y[:,0,:,:],y[:,3,:,:],y[:,12,:,:],y[:,13,:,:],y[:,14,:,:]])
R = layers.Concatenate()([y[:,0,:,:],y[:,4,:,:],y[:,15,:,:],y[:,16,:,:],y[:,17,:,:]])
P = layers.Concatenate()([y[:,0,:,:],y[:,5,:,:],y[:,18,:,:],y[:,19,:,:],y[:,20,:,:]])
print("T:")
print(R[0][50][8]) 
print(T)

Creating one model

In [0]:
#model = create_model(0.0001,  128, 10*128)
model.fit([T,I,M,R,P], y_train, epochs = 10, batch_size=5)

Epoch 1/10
190/190 [==============================] - 28s 149ms/step - loss: 2.9262e-04 - accuracy: 1.0000
Epoch 2/10
190/190 [==============================] - 28s 148ms/step - loss: 2.9643e-04 - accuracy: 1.0000
Epoch 3/10
190/190 [==============================] - 28s 148ms/step - loss: 2.5747e-04 - accuracy: 1.0000
Epoch 4/10
190/190 [==============================] - 28s 148ms/step - loss: 2.2810e-04 - accuracy: 1.0000
Epoch 5/10
190/190 [==============================] - 28s 149ms/step - loss: 2.0477e-04 - accuracy: 1.0000
Epoch 6/10
190/190 [==============================] - 28s 148ms/step - loss: 1.9624e-04 - accuracy: 1.0000
Epoch 7/10
190/190 [==============================] - 28s 147ms/step - loss: 1.8403e-04 - accuracy: 1.0000
Epoch 8/10
190/190 [==============================] - 28s 149ms/step - loss: 1.5663e-04 - accuracy: 1.0000
Epoch 9/10
190/190 [==============================] - 28s 147ms/step - loss: 1.3663e-04 - accuracy: 1.0000
Epoch 10/10
190/190 [================

Testing one model

In [0]:
model.evaluate([Tout,Iout,Mout,Rout,Pout], y_test)

8/8 [==============================] - 1s 88ms/step - loss: 0.7528 - accuracy: 0.8400


[0.7527546286582947, 0.8399999737739563]

In [0]:
def create_model(): 
  max_len = 1151
  visible2 = layers.Input(shape=(max_len, 15), name='visible2')
  visible3 = layers.Input(shape=(max_len, 15), name='visible3')
  visible4 = layers.Input(shape=(max_len, 15), name='visible4')
  visible5 = layers.Input(shape=(max_len, 15), name='visible5')
  visible6 = layers.Input(shape=(max_len, 15), name='visible6')

  branch_outputs = []

  #Thumb
  mask = layers.Masking(mask_value=-9.0,input_shape=(max_len,15))(visible2)
  out = layers.GRU(10, input_shape=(max_len,15), return_sequences=True)(mask)
  branch_outputs.append(out)

  #Index
  mask = layers.Masking(mask_value=-9.0,input_shape=(max_len,15))(visible3)
  out = layers.GRU(10, input_shape=(max_len,15), return_sequences=True)(mask)
  branch_outputs.append(out)

  #Middle
  mask = layers.Masking(mask_value=-9.0,input_shape=(max_len,15))(visible4)
  out = layers.GRU(10, input_shape=(max_len,15), return_sequences=True)(mask)
  branch_outputs.append(out)

  #Ring
  mask = layers.Masking(mask_value=-9.0,input_shape=(max_len,15))(visible5)
  out = layers.GRU(10, input_shape=(max_len,15), return_sequences=True)(mask)
  
  branch_outputs.append(out)

  #Pinky
  mask = layers.Masking(mask_value=-9.0,input_shape=(max_len,15))(visible6)
  out = layers.GRU(10, input_shape=(max_len,15), return_sequences=True)(mask)
  branch_outputs.append(out)

  merge = layers.concatenate(branch_outputs)
  temp = layers.GRU(100, input_shape=(max_len,100))(merge)
  output = layers.Dense(45, activation='softmax', name='output')(temp)
  model = models.Model(inputs=[visible2, visible3, visible4, visible5, visible6], outputs=output)
  model.summary()
  adam = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=adam, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
  return model

In [0]:
model = create_model()
model.fit([T,I,M,R,P], y_train, epochs = 50, batch_size=10)

Testing different learning rates

In [0]:
acc_data = np.zeros(10)
loss_data = np.zeros(10)

for k in range(0,4):
  tot_acc, tot_loss = 0, 0
  model = create_model(0.0001*(10**k), 128, 1280) # possible LRs: 0.0001, 0.001, 0.01, 0.1
  model.fit([T,I,M,R,P], y_train, epochs = 50, batch_size=5)
  loss, acc = model.evaluate([Tout,Iout,Mout,Rout,Pout], y_test)
  acc_data[k] = acc
  loss_data[k] = loss

print("accuracy:")
print(acc_data)
print("loss:")
print(loss_data)

Epoch 1/50
190/190 [==============================] - 15s 77ms/step - loss: 3.5012 - accuracy: 0.0989
Epoch 2/50
190/190 [==============================] - 15s 77ms/step - loss: 2.7631 - accuracy: 0.2453
Epoch 3/50
190/190 [==============================] - 15s 77ms/step - loss: 2.2531 - accuracy: 0.3716
Epoch 4/50
190/190 [==============================] - 15s 78ms/step - loss: 1.9212 - accuracy: 0.4484
Epoch 5/50
190/190 [==============================] - 15s 77ms/step - loss: 1.6498 - accuracy: 0.5179
Epoch 6/50
190/190 [==============================] - 15s 78ms/step - loss: 1.4608 - accuracy: 0.5674
Epoch 7/50
190/190 [==============================] - 15s 77ms/step - loss: 1.4127 - accuracy: 0.5853
Epoch 8/50
190/190 [==============================] - 15s 77ms/step - loss: 1.1113 - accuracy: 0.6705
Epoch 9/50
190/190 [==============================] - 15s 77ms/step - loss: 0.9684 - accuracy: 0.6884
Epoch 10/50
190/190 [==============================] - 15s 78ms/step - loss: 0.944

Different LSTM sizes

In [0]:
acc_data = np.zeros(10)
loss_data = np.zeros(10)

for k in range(0,4):
  tot_acc, tot_loss = 0, 0
  model = create_model(0.001, 2**(3+k), 10*(2**(3+k))) # possible small LSTM sizes: 8, 16, 32, 64
  model.fit([T,I,M,R,P], y_train, epochs = 50, batch_size=50)
  loss, acc = model.evaluate([Tout,Iout,Mout,Rout,Pout], y_test)
  acc_data[k] = acc
  loss_data[k] = loss

print("accuracy:")
print(acc_data)
print("loss:")
print(loss_data)

Epoch 1/50
19/19 [==============================] - 2s 84ms/step - loss: 3.7983 - accuracy: 0.0347
Epoch 2/50
19/19 [==============================] - 2s 84ms/step - loss: 3.6796 - accuracy: 0.0874
Epoch 3/50
19/19 [==============================] - 2s 84ms/step - loss: 3.4123 - accuracy: 0.1137
Epoch 4/50
19/19 [==============================] - 2s 83ms/step - loss: 3.2462 - accuracy: 0.1547
Epoch 5/50
19/19 [==============================] - 2s 84ms/step - loss: 3.2717 - accuracy: 0.1284
Epoch 6/50
19/19 [==============================] - 2s 84ms/step - loss: 3.0830 - accuracy: 0.1874
Epoch 7/50
19/19 [==============================] - 2s 85ms/step - loss: 2.9889 - accuracy: 0.2021
Epoch 8/50
19/19 [==============================] - 2s 84ms/step - loss: 2.9601 - accuracy: 0.2063
Epoch 9/50
19/19 [==============================] - 2s 82ms/step - loss: 2.8248 - accuracy: 0.2579
Epoch 10/50
19/19 [==============================] - 2s 82ms/step - loss: 2.7416 - accuracy: 0.2695
Epoch 11/

In [0]:
print("accuracy:")
print(acc_data)
print("loss:")
print(loss_data)

Different batch sizes

In [0]:
acc_data = np.zeros(10)
loss_data = np.zeros(10)

for k in range(0,3):
  tot_acc, tot_loss = 0, 0
  model = create_model(0.001, 16, 160)
  model.fit([T,I,M,R,P], y_train, epochs = 50, batch_size=5*(2**k)) # batch sizes 5, 10, 20 (test 50 and 100 separately)
  loss, acc = model.evaluate([Tout,Iout,Mout,Rout,Pout], y_test)
  acc_data[k] = acc
  loss_data[k] = loss

print("accuracy:")
print(acc_data)
print("loss:")
print(loss_data)

Epoch 1/50
190/190 [==============================] - 9s 47ms/step - loss: 3.5972 - accuracy: 0.0926
Epoch 2/50
190/190 [==============================] - 9s 47ms/step - loss: 3.0466 - accuracy: 0.1874
Epoch 3/50
190/190 [==============================] - 9s 48ms/step - loss: 2.5751 - accuracy: 0.2895
Epoch 4/50
190/190 [==============================] - 9s 49ms/step - loss: 2.4042 - accuracy: 0.3221
Epoch 5/50
190/190 [==============================] - 9s 47ms/step - loss: 2.2444 - accuracy: 0.3821
Epoch 6/50
190/190 [==============================] - 9s 48ms/step - loss: 2.0080 - accuracy: 0.4484
Epoch 7/50
190/190 [==============================] - 9s 48ms/step - loss: 1.9508 - accuracy: 0.4326
Epoch 8/50
190/190 [==============================] - 9s 48ms/step - loss: 1.6908 - accuracy: 0.5053
Epoch 9/50
190/190 [==============================] - 9s 48ms/step - loss: 1.5145 - accuracy: 0.5779
Epoch 10/50
190/190 [==============================] - 9s 48ms/step - loss: 1.4853 - accura

In [0]:
acc_data = np.zeros(10)
loss_data = np.zeros(10)

for k in range(0,2):
  tot_acc, tot_loss = 0, 0
  model = create_model(0.001, 16, 160)
  model.fit([T,I,M,R,P], y_train, epochs = 50, batch_size=50*(2**k)) # batch sizes 50 and 100
  loss, acc = model.evaluate([Tout,Iout,Mout,Rout,Pout], y_test)
  acc_data[k] = acc
  loss_data[k] = loss

print("accuracy:")
print(acc_data)
print("loss:")
print(loss_data)

Epoch 1/50
19/19 [==============================] - 2s 93ms/step - loss: 3.7387 - accuracy: 0.0716
Epoch 2/50
19/19 [==============================] - 2s 93ms/step - loss: 3.4221 - accuracy: 0.1011
Epoch 3/50
19/19 [==============================] - 2s 94ms/step - loss: 3.0505 - accuracy: 0.1853
Epoch 4/50
19/19 [==============================] - 2s 95ms/step - loss: 2.7331 - accuracy: 0.2579
Epoch 5/50
19/19 [==============================] - 2s 96ms/step - loss: 2.4593 - accuracy: 0.3158
Epoch 6/50
19/19 [==============================] - 2s 97ms/step - loss: 2.3157 - accuracy: 0.3737
Epoch 7/50
19/19 [==============================] - 2s 102ms/step - loss: 2.1410 - accuracy: 0.4137
Epoch 8/50
19/19 [==============================] - 2s 94ms/step - loss: 1.9688 - accuracy: 0.4621
Epoch 9/50
19/19 [==============================] - 2s 96ms/step - loss: 1.9081 - accuracy: 0.4726
Epoch 10/50
19/19 [==============================] - 2s 98ms/step - loss: 1.7789 - accuracy: 0.5189
Epoch 11